# Building a Bitstream

In this step we will be creating a FPGA Bitstream with your HLS core from **[Creating a Vivado HLS Core](2-Creating-A-Vivado-HLS-Core.ipynb)**. We will be using Vivado to create a block diagram, export it as a `.tcl` file, and compiling it into a `.bit` file.

PYNQ uses this `.tcl` file to match FPGA cores to existing drivers. 

To skip this notebook, copy the contents of the `~/PYNQ-HLS/pynqhls/stream/` directory to `~/PYNQ-HLS/tutorial/pynqhls/stream` by running the following command on your host computer:

``` bash
    cp ~/PYNQ-HLS/pynqhls/stream/* ~/PYNQ-HLS/tutorial/pynqhls/stream/
```

This command command can be run in Cygwin, or in a Bash shell in Linux.

## Adding Vivado to your Executable Path

If you have not already, add Vivado to your executable path. In Cygwin, you can do this by running:

``` bash
source C:/Xilinx/Vivado/2017.1/settings64.sh
```

Or on Linux: 

``` bash
source /opt/Xilinx/Vivado/2017.1/settings64.sh
```

## Creating a Vivado Project

We will begin by creating a Vivado project. On your host computer, navigate to the `tutorial/pynqhls/stream` folder of the PYNQ-HLS repository using your terminal:

```bash
cd ~/PYNQ-HLS/tutorial/pynqhls/stream
```



In this directory we have provided a makefile that will: 

1. Create a Vivado project for the PYNQ board
2. Add the `filt1d` IP path to the Vivado project
3. Create a block diagram using `stream.tcl`
4. Instantiate a correctly-configured ZYNQ ARM Processing System 
5. Compile the project into a FPGA bitstream

To run the makefile, run the following command from from the directory above.

``` bash
make
```

This will perform the steps above. When this is complete run the following command: 

```bash
vivado stream/stream.xpr
```

You should now see the following window: 


<img src="pictures/vivado_stream_splash.png" alt="Stream Project in Vivado" style="width: 768px;"/>

# Adding IP to the Block Diagram

Once the project has been created, we must add the HLS core, a DMA engine, and communication cores. 

Open the block diagram by expanding the top level and double clicking on `stream_i`. You will see the window shown below: 

<img src="pictures/vivado_stream_bd.png" alt="Initial Block Diagram in Vivado" style="width: 768px;"/>

Click on the `+` icon to add cores to your block design. In the search box that appears, type `filt1d`. You should see the following window:

<img src="pictures/vivado_stream_core.png" alt="The filt1d HLS core in Vivado" style="width: 256px;"/>

Double click on "Simple 1-D Filter" to add the core to the block diagram. **Change the name to `filt1d` using the properties sidebar.** You should see a block diagram similar to the following: 

<img src="pictures/vivado_stream_bd_2.png" alt="The filt1d HLS core in the Stream Block Diagram Vivado" style="width: 768px;"/>

We will perform wiring later. Add the following cores to the block diagram, and name them accordingly using the process described above: 

- AXI DMA Engine (`hlsDmaEngine`)
    - This is the DMA Engine for transferring data between the ARM and cores in the FPGA fabric
- AXI Interconnect (`mmioInterconnect`)
    - This routes addressed configuration register writes that originate in the ARM Processing system
- AXI Interconnect (`dmaInterconnect`)
    - This routes data transfers between the DMA Engine and the DDR Memory of the ARM Processing System.  
- Processor System Reset (`porReset`)
    - This handles the sequencing of power-on-reset (POR)
- Processor System Reset (`userReset`)
    - This handle the sequencing of User-Driven Resets (From PYNQ)
- Concat (`irqConcat`)
    - This concatenates all IRQ Signals into a single bus.
- AXI Interrupt Controller (`plInterruptController`)
    - This is the endpoint for the concatenated bus described above. 

## Configuration

You will need to configure four IP Blocks: 

### AXI DMA Engine (`hlsDmaEngine`)

First, configure the DMA Engine. Double click to bring up the window below.

You should match the settings shown in the picture below:
- Enable Single AXI4 Data Interface
- Disable the Scatter Gather Engine
- Change Width of Buffer Length Register to 23

<img src="pictures/vivado_stream_dma_engine.png" alt="Configuring the DMA Engine in Vivado" style="width: 768px;"/>

### AXI Interconnect (`mmioInterconnect`)

Next, configure the first AXI Interconnect, named `mmioInterconnect`. 

You should match the settings shown in the picture below:
- Number of Slave Interfaces : 1
- Number of Master Interfaces : 3

<img src="pictures/vivado_stream_mmio_interconnect.png" alt="Configuring the MMIO Interconnect AXI Switch Engine in Vivado" style="width: 768px;"/>


### AXI Interconnect (`dmaInterconnect`)

Next, configure the first AXI Interconnect, named `dmaInterconnect`. 

You should match the settings shown in the picture below:
- Number of Slave Interfaces : 1
- Number of Master Interfaces : 1

<img src="pictures/vivado_stream_dma_interconnect.png" alt="Configuring the DMA Interconnect AXI Switch Engine in Vivado" style="width: 768px;"/>

### Concat (irqConcat)

Finally, configure the Concat block, named `irqConcat`.

You should match the settings shown in the picture below:

- Number of Ports: 3

<img src="pictures/vivado_stream_irq_concat.png" alt="Configuring the IRQ Concat Block in Vivado" style="width: 768px;"/>


## Wiring: 

For wiring, follow the directions below. You can change the visible layers (clocks, GPIO, interfaces, etc) by clicking the cog button in the right hand corner. 

### Clocks 

Connect all clocks to the `FCLK_CLK0` output of the ARM Processing System Block (processing_system_7_0) as shown below:

<img src="pictures/vivado_stream_bd_clocks.png" alt="Clocks in the Stream Block Diagram Vivado" style="width: 768px;"/>
    
### Resets

Using the block diagram editor, we will connect the available resets from the Reset Controllers. The Reset Controllers improve timing and ensure AXI-Standard resets. In general `interconnect_aresetn` connects to any `ARESETN` port on an AXI Interconnect, and `peripheral_aresetn` connects to any peripheral (like a DMA Engine, or HLS core). `peripheral_aresetn` must drive the reset port of the peripheral **AND** the corresponding reset port of AXI interconnect. This is shown in the list below:

The porReset controller is reset on Power-On-Reset from `FCLK0_RESET_N`. This controller drives anything that must be available after the bitstream is loaded. 

- **porReset**
    - Connect the `FCLK0_RESET_N` of processing_system_7_0 to ext_reset of porReset 
    - Connect `interconnect_aresetn` of porReset to:
        - `ARESETN` of `dmaInterconnect`
        - `ARESETN` of `mmioInterconnect`
    - Connect peripheral_aresetn of porReset to:
        - `SOO_ARESETN` of `dmaInterconnect`
        - `MO1_ARESETN` of `mmioInterconnect`
        - `axi_aresetn` of `hlsDmaEngine`
        - `MO2_ARESETN` of `mmioInterconnect`
        - `s_axi_aresetn` of `plInterruptController`
        
The userReset controller is driven by a GPIO output from the ARM Processing System. This allows the user to reset the HLS core. 
- **userReset**
    - Connect the `FCLK0_RESET_N` of `processing_system_7_0` to `aux_reset` of `userReset`
    - Expand `GPIO_0` (as in the number zero) of `processing_system_7_0` and connect `GPIO_O` (as in the alphabetical letter O) to `ext_reset` of `userReset`
        - This is shown in the IRQ & Other figure
    - Connect `peripheral_aresetn` of `userReset` to: 
        - `ap_rst_n` of `filt1d`
        - `M00_ARESETN` of `mmioInterconnect`
    
<img src="pictures/vivado_stream_bd_resets.png" alt="Resets in the Stream Block Diagram Vivado" style="width: 768px;"/>

### Interrupts & Other: 

We use interrupts to notify the ARM Processing System about events in the programmable logic. Wire the Interrupt Outputs as shown in the following picture. 

- Connect `mm2s_introut` to `in0` of `irqConcat`
- Connect `s2mm_introut` to `in1` of `irqConcat`
- Connect `interrupt` of `filt1d` to `in2` of `irqConcat `
- Connect `dout` of `irqConcat` to `intr` of `plInterruptController`
- Connect `irq` of `plInterruptController` to `IRQ_F2P`

<img src="pictures/vivado_stream_bd_other.png" alt="IRQs and Other wires in the Stream Block Diagram Vivado" style="width: 768px;"/>

### AXI Memory Mapped Interfaces

AXI Memory Mapped interfaces are an addressed bus standard. Connect the AXI interfaces as shown in the picture below: 

- Connect `M_AXI_GP0` of `processing_system_7_0` to `S00_AXI` of `mmioInterconnect`
    - Connect `M00_AXI` of `mmioInterconnect` to `s_axi_CTRL` of `filt1d`
    - Connect `M01_AXI` of `mmioInterconnect` to `S_AXI_LITE` of `hlsDmaEngine`
    - Connect `M02_AXI` of `mmioInterconnect` to `s_axi` of `plInterruptController`
- Connect `M_AXI` of `hlsDmaEngine` to `S00_AXI` of `dmaInterconnect`
- Connect `M00_AXI` of `dmaInterconnect` to `S_AXI_HP0` of `processing_system_7_0`

<img src="pictures/vivado_stream_bd_axim.png" alt="AXI Memory-Mapped Interfaces in the Stream Block Diagram Vivado " style="width: 768px;"/>

Once these are connected, you will need to assign the address map. Click on the Address Editor tab and copy the settings from below: 

<img src="pictures/vivado_stream_bd_addresses.png" alt="AXI Memory-Mapped Interfaces in the Stream Block Diagram Vivado" style="width: 768px;"/>

The address shown above are significant. On ZYNQ, the AXI port M_AXI_GP0 from the ARM Processing System to the FPGA Programmable Logic is mapped to the physical address range `0x4000_0000` to `0x8000_0000`. The `filt1d`, `hlsDmaEngine`, and `plInterruptContoller` are all mapped into this address space in contiguous 4K blocks.

You can find more information about the ZYNQ address map [here](https://www.xilinx.com/support/documentation/user_guides/ug585-Zynq-7000-TRM.pdf)

On the AXI Slave interface for the ARM Processing System, the kernel provides memory for the Contiguous Memory Allocator in a 512 MB chunk starting at Address `0x0000_0000`. Thus, the DMA engine must have an address space from `0x0000_0000` to `0x1FFF_FFFF` (512 MB).

Once these addresses have been assigned, they will automagically be discovered in PYNQ from the `.tcl` file. 

### AXI Streaming Interfaces

AXI Streaming interfaces are a non-addressed bus standard. Connect the AXI interfaces as shown in the picture below: 

<img src="pictures/vivado_stream_bd_axis.png" alt="AXI Streaming Interfaces in the Stream Block Diagram Vivado " style="width: 768px;"/>

## Checking your Progress
Once you have completed the wiring, you can validate your block design by selecting **Tools** -> **Validate Design** from the dropdown menu. 

If your design passes, then you can proceed! 

Save before proceeding to the next step.

## Exporting your Block Design

As the final step of this tutorial, we will export and compile your block design. 

Export your block design by clicking **File ** -> **Export...** -> **Export Block Design**. You will be presented with the following window: 

<img src="pictures/vivado_stream_bd_export.png" alt="Exporting the Stream Block Diagram Vivado 2017.1 " style="width: 512px;"/>

Remove the highlighted text and **OVERWRITE** the existing `~/PYNQ-HLS/tutorial/pynqhls/stream/stream.tcl` file.

When finished, close Vivado


## Compiling your Block Design

When Vivado has closed, use the makefile to rebuild the `stream.bit` bitstream using the following commands: 

```bash
cd ~/PYNQ-HLS/tutorial/pynqhls/stream/
make
```

This process can take up to 15 minutes. If your compilation completes without errors, proceeed to the next notebook, **[Using an HLS core in PYNQ](4-Using an HLS Core in PYNQ.ipynb)**.
